In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
submission_df = pd.read_csv("y_preds.csv")
df_test = pd.read_csv('test_amended.csv')

In [3]:
df_test_non = pd.read_csv('test.csv')

In [4]:
# one of the interpolations was placed below 0.7 which makes the upload break
df_test.iloc[889,1] = 0.75

In [5]:
df_test

,galactic year,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",...,galaxy_UGCA 292,galaxy_UGCA 438 (ESO 407-018),galaxy_UGCA 86,galaxy_UGCA 92,galaxy_Ursa Major I Dwarf (UMa I dSph),galaxy_Ursa Major II Dwarf,galaxy_Ursa Minor Dwarf,galaxy_Virgo I,galaxy_Willman 1,"galaxy_Wolf-Lundmark-Melotte (WLM, DDO 221)"
0,1007012,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,232.621842,...,0,0,0,0,0,0,0,0,0,0
1,1007012,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,247.580771,...,0,0,0,0,0,0,0,0,0,0
2,1008016,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,249.798771,...,0,0,0,0,0,0,0,0,0,0
3,1007012,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,211.505060,...,0,0,0,0,0,0,0,0,0,0
4,1013042,0.588274,55.428320,23959.704016,0.520579,10.392416,6.374637,0.530676,0.580418,234.721069,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,1016064,0.859797,75.291914,18170.157548,0.707996,11.154777,4.190958,0.701760,0.540954,212.874869,...,0,0,0,0,0,0,0,0,0,0
886,1016064,0.912304,72.601017,19767.681597,0.715522,13.126625,10.552630,0.732641,0.622619,190.209811,...,0,0,0,0,0,0,0,0,0,0
887,1016064,1.090993,88.276857,31700.561656,0.957858,19.425120,11.120741,1.092724,0.974172,128.115014,...,0,0,0,0,0,0,0,0,0,0
888,1016064,1.005144,82.100572,25561.963151,0.751660,16.198364,13.756741,0.819249,0.884783,156.160418,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# creating the energy allocation - mean allocation

# amount for galaxies with amount below 0.7
bottom10 = 50000*0.1/len(df_test[df_test['existence expectancy index']<0.7])
topenergy = 50000*0.9/len(df_test[~(df_test['existence expectancy index']<0.7)])
print(bottom10,topenergy)

75.75757575757575 54.61165048543689


In [7]:
energy_allocation = df_test['existence expectancy index'].apply(lambda x: bottom10 if x<0.7 else topenergy)

In [8]:
submission_df['opt_pred'] = energy_allocation

In [9]:
submission_df.reset_index(inplace=True)
submission_df.to_csv('submission1.csv',index=False)

In [42]:
submission_df.sort_values("pred")

,index,pred,opt_pred,index_increase
226,226,0.014885,54.611650,2.446761
49,49,0.016447,75.757576,3.332686
37,37,0.019692,75.757576,3.217386
26,26,0.019692,75.757576,3.217386
34,34,0.019692,75.757576,3.217386
...,...,...,...,...
705,705,0.455698,54.611650,0.773811
800,800,0.455698,54.611650,0.773811
801,801,0.455698,54.611650,0.773811
583,583,0.680323,54.611650,0.620438


## Playing around with optimisation

In [11]:
submission_df['index_increase'] = submission_df.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000

In [13]:
submission_df.index_increase.sum()

1597.651362778016

In [16]:
test1 = submission_df.sort_values("pred")

In [15]:
df_test[df_test['existence expectancy index']<0.7].index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65],
           dtype='int64')

In [41]:
test1.opt_pred.iloc[:500] = 100
test1.opt_pred.iloc[500:] = 0
test1['index_increase'] = test1.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test1.index_increase.sum()

/Users/lukebetham/anaconda3/envs/main_env/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1811.8798144030254

In [32]:
test1[test1.index.isin(df_test[df_test['existence expectancy index']<0.7].index)]

,index,pred,opt_pred,index_increase
49,49,0.016447,100.0,4.399145
37,37,0.019692,100.0,4.246950
26,26,0.019692,100.0,4.246950
34,34,0.019692,100.0,4.246950
27,27,0.021095,100.0,4.187013
...,...,...,...,...
55,55,0.043200,100.0,3.520886
24,24,0.043200,100.0,3.520886
61,61,0.046160,100.0,3.456911
33,33,0.058699,0.0,0.000000


In [35]:
test1[499:501]

,index,pred,opt_pred,index_increase
723,723,0.055550,100.0,3.277488
536,536,0.056333,0.0,0.000000


In [37]:
# check if there are decreasing returns - it would seem that there aren't
(100*(-np.log(0.055550+0.01)+3)**2/1000)-(99*(-np.log(0.055550+0.01)+3)**2/1000)-(1*(-np.log(0.056333+0.01)+3)**2/1000)

0.00013581836830876132

In [39]:
test1.opt_pred.iloc[500:] = 5
test1.opt_pred.iloc[:500] = (50000 - (len(test1.opt_pred.iloc[500:])*5))/500

test1['index_increase'] = test1.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test1.index_increase.sum()

/Users/lukebetham/anaconda3/envs/main_env/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1792.1555657256326

In [40]:
test1

,index,pred,opt_pred,index_increase
226,226,0.014885,96.1,4.305560
49,49,0.016447,96.1,4.227579
37,37,0.019692,96.1,4.081319
26,26,0.019692,96.1,4.081319
34,34,0.019692,96.1,4.081319
...,...,...,...,...
705,705,0.455698,5.0,0.070847
800,800,0.455698,5.0,0.070847
801,801,0.455698,5.0,0.070847
583,583,0.680323,5.0,0.056805


In [43]:
test1.opt_pred.iloc[:450] = 100
test1.opt_pred.iloc[450:550] = 50
test1.opt_pred.iloc[450:550] = 0


test1['index_increase'] = test1.opt_pred*(-np.log(submission_df.pred+0.01)+3)**2/1000
test1.index_increase.sum()

/Users/lukebetham/anaconda3/envs/main_env/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1646.6430951262373